In [9]:
#from mlflow.tracking import MlflowClient
from mlflow import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///wine_pred.db"

In [16]:
# Initialize the client (tracking URI is optional if the default is being used)
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# List experiments using the client method
experiments = client.search_experiments()
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}, Name: {experiment.name}, Artifact Location: {experiment.artifact_location}")

Experiment ID: 1, Name: wine-quality-prediction, Artifact Location: /workspaces/mlops_wine/mlruns/1
Experiment ID: 0, Name: Default, Artifact Location: mlflow-artifacts:/0
